In [7]:
%reload_ext autoreload
%autoreload 2

In [418]:
from parse_hocr_files import make_hocr_page
import republic_column_parser as column_parser
from elasticsearch import Elasticsearch
import republic_page_parser as page_parser
import republic_paragraph_parser as paragraph_parser
import republic_file_parser as file_parser

import copy

year = 1725
inventory_num = 3767
base_config = {
    "inventory_num": inventory_num,
    "base_dir": "../../../Data/Projects/REPUBLIC/hocr/",
    "page_index": "republic_hocr_pages",
    "page_doc_type": "page",
    "scan_index": "republic_hocr_scans",
    "scan_doc_type": "scan",
    "tiny_word_width": 15, # pixel width
    "avg_char_width": 20,
    "remove_tiny_words": True,
    "remove_line_numbers": False,
    "normal_scan_width": 4840
}


def set_config_inventory_num(base_config, inventory_num):
    config = copy.deepcopy(base_config)
    config["inventory_num"] = inventory_num
    config["data_dir"] = config["base_dir"] + "{}/".format(inventory_num)
    return config

inventory_config = set_config_inventory_num(base_config, inventory_num)
print(inventory_config)
scan_files = file_parser.get_files(inventory_config["data_dir"])
print("Number of scan files:", len(scan_files))

scan_file = scan_files[10]
print(scan_file)

data_dir = "/Users/marijnkoolen/Data/Projects/REPUBLIC/3234"

# scan num 751 is index start
# scan num 6 is resolution start

scan_file = scan_files[5]
scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
#print(scan_hocr["scan_type"])
#print(scan_hocr["page_box"])
if "double_page" in scan_hocr["scan_type"]:
    pages_hocr = column_parser.parse_double_page_scan(scan_hocr, column_config)
    for page_hocr in pages_hocr:
        print(page_hocr["page_num"], page_parser.get_page_type(page_hocr, debug=False))
        page_hocr["page_type"] += [page_parser.get_page_type(page_hocr, debug=False)]
        for column in page_hocr["columns"]:
            del column["lines"]
        #print(json.dumps(page_hocr, indent=2))

{'inventory_num': 3767, 'base_dir': '../../../Data/Projects/REPUBLIC/hocr/', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'year': 3767, 'data_dir': '../../../Data/Projects/REPUBLIC/hocr/3767/'}
Number of scan files: 774
{'scan_num': 11, 'inventory_num': 3767, 'inventory_year': 1712, 'inventory_period': ['1712-01-01', '1712-12-31'], 'filepath': '../../../Data/Projects/REPUBLIC/hocr/3767/NL-HaNA_1.01.02_3767_0011.hocr'}
11 empty_page
12 resolution_page


In [377]:
import republic_column_parser as column_parser

column_config = {
    "avg_char_width": 20,
    "word_conf_threshold": 10,
    "column_gap_threshold": 50,
    "gap_pixel_freq_ratio": 0.75,
    "fulltext_char_threshold": 0.5,
    "fulltext_words_threshold": 15,
    "filter_words": ["|", "{", "$"]
}


scan_file = scan_files[196]
scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
print(scan_hocr["scan_num"], scan_hocr["scan_type"])
if "double_page" in scan_hocr["scan_type"]:
    pages_hocr = column_parser.parse_double_page_scan(scan_hocr, column_config)
    
# TO DO
# - determine page side and page number (scan * 2 for even scan * 2 - 1 for odd)

for page_hocr in pages_hocr:
    print("PAGE HOCR")
    #print(page_hocr)
    print(page_hocr["page_type"])
    columns_hocr = page_hocr["columns"]
    print("NUM COLUMNS:", len(columns_hocr))
    for column in columns_hocr:
        for line_index, line in enumerate(column["lines"]):
            print("column {}, line {} ({}, {}) ({}, {}):".format(line["column_num"], line_index+1, line["left"], line["right"], line["top"], line["bottom"]), 
                  " ".join([word["word_text"] for word in line["words"]]))

197 ['normal', 'double_page']
num_columns: 2
num_columns: 4
PAGE HOCR
['normal', 'double_column', 'resolution_page']
NUM COLUMNS: 2
column 1, line 1 (476, 1513) (341, 393): Den 1, April, ( 380 >
column 1, line 2 (478, 1357) (401, 449): digen Oorlogh, gier alleen Sr. David Mitchel,
column 1, line 3 (480, 1357) (464, 496): maer oock Sr. James Wishart op ordres van
column 1, line 4 (478, 1358) (504, 542): hare Majeltejt ‘naer Hollandt zyn gegaen, om
column 1, line 5 (477, 1361) (558, 583): met de Heeren Staten Generael, of der felver
column 1, line 6 (476, 1361) (605, 642): Gedeputeerdens te adjufteren de quote van ha-
column 1, line 7 (474, 1360) (659, 692): re Schepen van Oorlogh om in conjunêtie met
column 1, line 8 (475, 1362) (704, 735): die van hare Majelteyt te ageren, en het gecne
column 1, line 9 (474, 1362) (748, 791): volgens dele behandelingen was gedaen, isaen
column 1, line 10 (474, 1365) (796, 838): hare Majefteyts Principael-Sccrerariflen van Stact
column 1, line 11 (474, 

In [414]:
column_config = {
    "avg_char_width": 20,
    "word_conf_threshold": 10,
    "column_gap_threshold": 40,
    "gap_pixel_freq_ratio": 0.75,
    "fulltext_char_threshold": 0.4,
    "fulltext_words_threshold": 15,
    "filter_words": ["|", "{", "$"]
}

for scan_file in scan_files[5:230]:
    scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
    if "double_page" in scan_hocr["scan_type"]:
        print("double page scan:", scan_hocr["scan_num"], scan_hocr["scan_type"])
        pages_hocr = column_parser.parse_double_page_scan(scan_hocr, column_config)
    else:
        print("NOT DOUBLE PAGE:", scan_hocr["scan_num"], scan_hocr["scan_type"])
        continue
    #columns_hocr = split_lines_on_columns(dp_hocr, columns_starts)
    for page_hocr in pages_hocr:
        columns_hocr = page_hocr["columns"]
        if len(columns_hocr) != 2:
            print("page type:", page_hocr["page_type"], "\tnum columns:", len(columns_hocr))



double page scan: 6 ['normal', 'double_page']
page type: ['special', 'no_column', 'empty_page'] 	num columns: 0
double page scan: 7 ['normal', 'double_page']
double page scan: 8 ['normal', 'double_page']
double page scan: 9 ['normal', 'double_page']
page type: ['special', 'multi_column', 'index_page'] 	num columns: 3
double page scan: 10 ['normal', 'double_page']
double page scan: 11 ['normal', 'double_page']
double page scan: 12 ['normal', 'double_page']
double page scan: 13 ['normal', 'double_page']
double page scan: 14 ['normal', 'double_page']
double page scan: 15 ['normal', 'double_page']
double page scan: 16 ['normal', 'double_page']
double page scan: 17 ['normal', 'double_page']
double page scan: 18 ['normal', 'double_page']
double page scan: 19 ['normal', 'double_page']
double page scan: 20 ['normal', 'double_page']
double page scan: 21 ['normal', 'double_page']
double page scan: 22 ['normal', 'double_page']
page type: ['special', 'multi_column', 'index_page'] 	num columns: 3
d

double page scan: 173 ['normal', 'double_page']
double page scan: 174 ['normal', 'double_page']
double page scan: 175 ['normal', 'double_page']
double page scan: 176 ['normal', 'double_page']
double page scan: 177 ['normal', 'double_page']
double page scan: 178 ['normal', 'double_page']
double page scan: 179 ['normal', 'double_page']
double page scan: 180 ['normal', 'double_page']
double page scan: 181 ['normal', 'double_page']
double page scan: 182 ['normal', 'double_page']
double page scan: 183 ['normal', 'double_page']
double page scan: 184 ['normal', 'double_page']
double page scan: 185 ['normal', 'double_page']
double page scan: 186 ['normal', 'double_page']
double page scan: 187 ['normal', 'double_page']
double page scan: 188 ['normal', 'double_page']
double page scan: 189 ['normal', 'double_page']
double page scan: 190 ['normal', 'double_page']
double page scan: 191 ['normal', 'double_page']
double page scan: 192 ['normal', 'double_page']
double page scan: 193 ['normal', 'double

In [426]:
import republic_elasticsearch as rep_es

def index_scan(es, scan_hocr, config):
    doc = rep_es.create_es_scan_doc(scan_hocr)
    es.index(index=config["scan_index"], doc_type=config["scan_doc_type"], id=scan_hocr["scan_id"], body=doc)
    

def index_page(es, page_hocr, config):
    doc = rep_es.create_es_page_doc(page_hocr)
    es.index(index=config["page_index"], doc_type=config["page_doc_type"], id=page_hocr["page_id"], body=doc)
    

def parse_inventory(inventory_num, base_config, es):
    inventory_config = set_config_inventory_num(base_config, inventory_num)
    print(inventory_config)
    scan_files = file_parser.get_files(inventory_config["data_dir"])
    for scan_file in scan_files:
        scan_hocr = page_parser.get_scan_hocr(scan_file, inventory_config)
        if "double_page" in scan_hocr["scan_type"]:
            print("double page scan:", scan_hocr["scan_num"], scan_hocr["scan_type"])
            pages_hocr = column_parser.parse_double_page_scan(scan_hocr, column_config)
            for page_hocr in pages_hocr:
                index_page(es, page_hocr, inventory_config)
        else:
            print("NOT DOUBLE PAGE:", scan_hocr["scan_num"], scan_hocr["scan_type"])
            index_scan(es, scan_hocr, inventory_config)
            continue

es = Elasticsearch()

inventory_num = 3768
parse_inventory(inventory_num, base_config, es)


{'inventory_num': 3768, 'base_dir': '../../../Data/Projects/REPUBLIC/hocr/', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'data_dir': '../../../Data/Projects/REPUBLIC/hocr/3768/'}
NOT DOUBLE PAGE: 1 ['special', 'small']
double page scan: 2 ['normal', 'double_page']
double page scan: 3 ['normal', 'double_page']
double page scan: 4 ['normal', 'double_page']
double page scan: 5 ['normal', 'double_page']
double page scan: 6 ['normal', 'double_page']
double page scan: 7 ['normal', 'double_page']
double page scan: 8 ['normal', 'double_page']
double page scan: 9 ['normal', 'double_page']
double page scan: 10 ['normal', 'double_page']
double page scan: 11 ['normal', 'double_page']
double page scan: 12 ['normal', 'double_page']
double page scan: 13 ['normal', 'double_page']


KeyboardInterrupt: 

In [ ]:


#rep_es.delete_es_index(base_config["page_index"])
#rep_es.delete_es_index(base_config["scan_index"])

inventory_numbers = [
    3767,
    3768,
    3769,
    3783,
    3796,
    3798,
    3853
]

for inventory_num in inventory_numbers:
    parse_inventory(inventory_num, base_config, es)



exists, deleting
exists, deleting
{'inventory_num': 3767, 'base_dir': '../../../Data/Projects/REPUBLIC/hocr/', 'page_index': 'republic_hocr_pages', 'page_doc_type': 'page', 'scan_index': 'republic_hocr_scans', 'scan_doc_type': 'scan', 'tiny_word_width': 15, 'avg_char_width': 20, 'remove_tiny_words': True, 'remove_line_numbers': False, 'normal_scan_width': 4840, 'data_dir': '../../../Data/Projects/REPUBLIC/hocr/3767/'}
NOT DOUBLE PAGE: 1 ['special', 'small']
double page scan: 2 ['normal', 'double_page']
double page scan: 3 ['normal', 'double_page']
double page scan: 4 ['normal', 'double_page']
double page scan: 5 ['normal', 'double_page']
double page scan: 6 ['normal', 'double_page']
double page scan: 7 ['normal', 'double_page']
double page scan: 8 ['normal', 'double_page']
double page scan: 9 ['normal', 'double_page']
double page scan: 10 ['normal', 'double_page']
double page scan: 11 ['normal', 'double_page']
double page scan: 12 ['normal', 'double_page']
double page scan: 13 ['normal

double page scan: 166 ['normal', 'double_page']
double page scan: 167 ['normal', 'double_page']
double page scan: 168 ['normal', 'double_page']
double page scan: 169 ['normal', 'double_page']
double page scan: 170 ['normal', 'double_page']
double page scan: 171 ['normal', 'double_page']
double page scan: 172 ['normal', 'double_page']
double page scan: 173 ['normal', 'double_page']
double page scan: 174 ['normal', 'double_page']
double page scan: 175 ['normal', 'double_page']
double page scan: 176 ['normal', 'double_page']
double page scan: 177 ['normal', 'double_page']
double page scan: 178 ['normal', 'double_page']
double page scan: 179 ['normal', 'double_page']
double page scan: 180 ['normal', 'double_page']
double page scan: 181 ['normal', 'double_page']
double page scan: 182 ['normal', 'double_page']
double page scan: 183 ['normal', 'double_page']
double page scan: 184 ['normal', 'double_page']
double page scan: 185 ['normal', 'double_page']
double page scan: 186 ['normal', 'double

double page scan: 337 ['normal', 'double_page']
double page scan: 338 ['normal', 'double_page']
double page scan: 339 ['normal', 'double_page']
double page scan: 340 ['normal', 'double_page']
double page scan: 341 ['normal', 'double_page']
double page scan: 342 ['normal', 'double_page']
double page scan: 343 ['normal', 'double_page']
double page scan: 344 ['normal', 'double_page']
double page scan: 345 ['normal', 'double_page']
double page scan: 346 ['normal', 'double_page']
double page scan: 347 ['normal', 'double_page']
double page scan: 348 ['normal', 'double_page']
double page scan: 349 ['normal', 'double_page']
double page scan: 350 ['normal', 'double_page']
double page scan: 351 ['normal', 'double_page']
double page scan: 352 ['normal', 'double_page']
double page scan: 353 ['normal', 'double_page']
double page scan: 354 ['normal', 'double_page']
double page scan: 355 ['normal', 'double_page']
double page scan: 356 ['normal', 'double_page']
double page scan: 357 ['normal', 'double

double page scan: 508 ['normal', 'double_page']
double page scan: 509 ['normal', 'double_page']
double page scan: 510 ['normal', 'double_page']
double page scan: 511 ['normal', 'double_page']
double page scan: 512 ['normal', 'double_page']
double page scan: 513 ['normal', 'double_page']
double page scan: 514 ['normal', 'double_page']
double page scan: 515 ['normal', 'double_page']
double page scan: 516 ['normal', 'double_page']
double page scan: 517 ['normal', 'double_page']
double page scan: 518 ['normal', 'double_page']
double page scan: 519 ['normal', 'double_page']
double page scan: 520 ['normal', 'double_page']
double page scan: 521 ['normal', 'double_page']
double page scan: 522 ['normal', 'double_page']
double page scan: 523 ['normal', 'double_page']
double page scan: 524 ['normal', 'double_page']
double page scan: 525 ['normal', 'double_page']
double page scan: 526 ['normal', 'double_page']
double page scan: 527 ['normal', 'double_page']
double page scan: 528 ['normal', 'double

double page scan: 679 ['normal', 'double_page']
double page scan: 680 ['normal', 'double_page']
double page scan: 681 ['normal', 'double_page']
double page scan: 682 ['normal', 'double_page']
double page scan: 683 ['normal', 'double_page']
double page scan: 684 ['normal', 'double_page']
double page scan: 685 ['normal', 'double_page']
double page scan: 686 ['normal', 'double_page']
double page scan: 687 ['normal', 'double_page']
double page scan: 688 ['normal', 'double_page']
double page scan: 689 ['normal', 'double_page']
double page scan: 690 ['normal', 'double_page']
double page scan: 691 ['normal', 'double_page']
double page scan: 692 ['normal', 'double_page']
double page scan: 693 ['normal', 'double_page']
double page scan: 694 ['normal', 'double_page']
double page scan: 695 ['normal', 'double_page']
double page scan: 696 ['normal', 'double_page']
double page scan: 697 ['normal', 'double_page']
double page scan: 698 ['normal', 'double_page']
double page scan: 699 ['normal', 'double

double page scan: 70 ['normal', 'double_page']
double page scan: 71 ['normal', 'double_page']
double page scan: 72 ['normal', 'double_page']
double page scan: 73 ['normal', 'double_page']
double page scan: 74 ['normal', 'double_page']
double page scan: 75 ['normal', 'double_page']
double page scan: 76 ['normal', 'double_page']
double page scan: 77 ['normal', 'double_page']
double page scan: 78 ['normal', 'double_page']
double page scan: 79 ['normal', 'double_page']
double page scan: 80 ['normal', 'double_page']
double page scan: 81 ['normal', 'double_page']
double page scan: 82 ['normal', 'double_page']
double page scan: 83 ['normal', 'double_page']
double page scan: 84 ['normal', 'double_page']
double page scan: 85 ['normal', 'double_page']
double page scan: 86 ['normal', 'double_page']
double page scan: 87 ['normal', 'double_page']
double page scan: 88 ['normal', 'double_page']
double page scan: 89 ['normal', 'double_page']
double page scan: 90 ['normal', 'double_page']
double page s